In [ ]:
## Please run this cell before the lecture --JL

#include <iostream>

# Object-oriented scientific programming with C++

Matthias Möller, Jonas Thies (Numerical Analysis, DIAM)

Lecture 3

## <center> Overview </center>
During last several lectures, we learned <b>polymorphism</b>, that is, inheritance of one class from another class
- Implementation of common functionality in **base class** (possibly realised as **abstract class** that cannot even be instantiated)
- Derivation of specialised class(es) from the base class that
    - implement the missing functionality (**pure virtual functions**)
    - override generic functionality by specialised variants (**virtual functions**)
    - reuse all other functionality from the base class

## <center>Overview</center>
Today, a more careful view on polymorphism
- **Static polymorphism**: static binding/method overloading
- **Dynamic polymorphism**: dynamic binding/method overriding
Auto functionality of C++11/14/17.

**Template meta programming**
- A powerful variant of static polymorphism

## <center> Quiz: Guess what happens </center>

In [ ]:
struct Base {
  virtual void hello() const = 0;
  virtual ~Base() {} // virtual destructor for proper cleanup
};

struct Derived : public Base {
  void hello() const override // Correct function name to override the base class method
  {
    std::cout << "Hallo\n";
  }
};

## <center> Quiz: Guess what happens </center>
Run the main function to verify your guess

In [ ]:
int main() {
  Derived d; // Create an instance of Derived
  Base* ptr = &d; // Pointer to Base type, pointing to Derived instance

  ptr->hello(); // Virtual dispatch ensures Derived::hello() is called

  return 0;
}

## <center> Quiz var1: Guess what happens </center>

<code>struct Base {
  virtual void hello() const = 0;
};
struct Derived : public Base {
  void h<span style=color:red;>e</span>llo() const
  {
    std::cout << “Hallo\n”;
  }
};</code>

## <center> Quiz var1: Guess what happens </center>
<code>struct Base {
  virtual void hello() const = 0;
};
struct Derived : public Base {
  void h<span style=color:red;>e</span>llo() const
  {
    std::cout << “Hallo\n”;
  }
};</code>

- Structurally correct in terms of C++ syntax. 
- <code>Derived</code> properly overrides the pure virtual function <code>hello()</code> from <code>Base</code>, allowing instances of <code>Derived</code> to be created and used polymorphically through pointers or references to <code>Base</code>.

## <center> Quiz var2: Guess what happens </center>

<code>struct Base {
  virtual void hello() const {
    std::cout &lt< “Hello\n”;
  }
};
struct Derived : public Base {
  void h<span style=color:red;>a</span>llo() const //Good luck with debugging
  {
    std::cout &lt< “Hallo\n”;
  }
};</code>


## <center> Quiz var2: Guess what happens </center>

<code>struct Base {
  virtual void hello() const {
    std::cout &lt< “Hello\n”;
  }
};
struct Derived : public Base {
  void h<span style=color:red;>a</span>llo() const //Good luck with debugging
  {
    std::cout &lt< “Hallo\n”;
  }
};</code>

- The Derived struct inherits from <code>Base</code> and introduces a new function <code>hallo()</code> that prints "<code>Hallo\n</code>" to the standard output. 
- However, <code>Derived</code> does not override the <code>hello()</code> function from Base; it simply provides an additional function.

## <center> The override keyword </center>
C++11 introduces the <span style=color:red;>override</span> keyword to explicitly state
that a function from the base class shall be <i>overridden</i>
<p><i>Modify the program below yourself to see the different with Quiz var2</i></p>

In [1]:
struct Base {
  virtual void hello() const = 0;
  virtual ~Base() {} // virtual destructor for proper cleanup
};
struct Derived : public Base {
  void hello() const override {std::cout << "Hallo\n";}
};
int main() {
  Derived d; // Create an instance of Derived
  d.hello(); // Calls the overridden hello() method in Derived
  Base* b = &d; // Since hello() is a virtual function, you can also call it through a Base pointer
  b->hello(); // This will also call Derived::hello()
  return 0;
} //NOTE: If the function to be overridden does not exist in the base class an error is thrown by the compiler

SyntaxError: invalid syntax (1569292975.py, line 1)

## <center>The override keyword</center>
The overriding function must <span style=color:red;>exactly match</span> the signature of the function from the base class

<code>struct Base {
virtual <span style=color:red;>void hello() const</span> { std::cout &lt&lt “Hello\n”; }
};</code>

<code>struct Derived : public Base {
<span style=color:red;>void hello() const override</span> { std::cout &lt&lt “Hallo\n”; }
};</code>

~~<code>struct Derived2 : public Base {
<span style=color:red;>void hello() override</span> { std::cout &lt&lt “Hallo\n”; }
};</code>~~

## <center> The final keyword</center>
C++11 introduces the <span style=color:red;>final</span> keyword to explicitly state that a
function from the base class <b>must not</b> be overridden

<code>struct Base {
virtual void hello() const <span style=color:red;>final</span> { std::cout &lt&lt “Hello\n”; } };</code>

If a derived class tries to override the function <span style=color:red;>hello</span> an error is thrown by the compiler

## <center>The override and final keywords</center>
The <span style=color:red>override</span> and <span style=color:red>final</span> keywords are often used together

<code>struct Base {
    <span style=color:blue;>// Pure virtual function must be overridden in a derived class</span>
    virtual void hello() const = 0;
  };
  struct Derived : public Base {
    <span style=color:blue;>// Override pure virtual function and make it final</span>
    void hello() const <span style=color:red;>override final</span>
    { std::cout &lt&lt “Hello\n”; }
  };
  struct Derived2 : public Derived {
    <span style=color:blue;>// Must not try to override function hello</span>
};</code>

## <center>Overloading vs. overriding</center>
<div id="left" style="float: left; width: 50%;">Implementation of method or operator with <span style=color:red;>identical</span> name but a <span style=color:blue;>different interface</span>, e.g.
<p><code><span style=color:blue;>double</span> <span style=color:red;>operator*</span>(<span style=color:blue;>const
Vector &amp; other</span>) <span style=color:blue;>const</span> {...}</code></p>
<h3>overloads</h3>
<p><code><span style=color:blue;>double</span> <span style=color:red;>operator*</span>(<span style=color:blue;>double
  other</span>) <span style=color:blue;>const</span> {...}</code></p>
</div>

<div id="right" style="float: right; width: 50%;">Implementation of method or operator with <span style=color:red;>identical</span> name and the <span style=color:blue;>same interface</span> in a derived class or structure, e.g.
<p><code>struct Derived : public Base {
    <span style=color:blue;>void</span> <span style=color:red;>hello</span>(...) const override {...}
};</code></p>
<h3>overrides</h3>
<p><code>struct Base {
    virtual <span style=color:blue;>void</span> <span style=color:red;>hello</span>(...) const {...};
};</code></p>    
</div>   

## <center>Overloading vs. overriding</center>
<img src="plots/lecture3_overloadingvsoverriding.png" width="700" >

## <center>Implementation time! Task: Calculator</center>
Write a class (or for demonstrating purposes a hierarchy of classes) that provide(s) a member function to calculate the **sum of two and three integer values,** respectively
- Use *static* polymorphism: method overloading
- Use *dynamic* polymorphism: method overriding

## <center>Static polymorphism</center>
<b>Method overloading</b> (at compile time), the <code>StaticCalc</code> has two member functions with <span style=color:red;>identical names</span> but <span style=color:blue;>different interface</span>; compiler can decide <b>at compile time</b> which of the two functions should be used

In [ ]:
struct StaticCalc {
    int sum(int a, int b) const {
        return a + b;
    }
    // Overloaded function to sum three integers
    int sum(int a, int b, int c) const { return sum(sum(a,b),c);}
};
int main() {
    StaticCalc statC;
    std::cout << "Static sum of two integers: " << statC.sum(1, 2) << std::endl;
    std::cout << "Static sum of three integers: " << statC.sum(1, 2, 3) << std::endl;
}

## <center>Static polymorphism</center>
<b>Method overloading</b> (not working!)

<code>struct StaticCalcBug {
<span style=color:red;>int sum</span>(int a, int b) { return a+b; }
<span style=color:red;>void sum</span>(int a, int b) { std::cout&lt&lt a+b &lt&ltstd::endl;}
};</code>
- Difference must be in the <b>interface of the arguments passed</b>. Same name with different parameter lists is allowed (either the number of parameters or their types must differ).
- You cannot have two functions that only differ by their <b>return type</b>. Because the return type could be cast into another type

## <center>Static polymorphism</center>
<b>Method overloading</b> (also not working!)

<code>struct StaticCalcBug2 {
     <span style=color:red;>int sum</span>(int <span style=color:purple;>a</span>, int <span style=color:purple;>b</span>) { return a+b; }
     <span style=color:red;>int sum</span>(int <span style=color:purple;>c</span>, int <span style=color:purple;>d</span>) { return c+d; }
  };</code>

- The interface only refers to the  <span style=color:blue;>types</span> and not to the  <span style=color:purple;>names</span>

## <center>Static polymorphism - Laaaaast! counter example</center>
<b>Method overloading</b> (still not working!)

<code>struct StaticCalcBugLast {
     <span style=color:red;>int sum</span>(int a, int b) { return a+b; }
     <span style=color:red;>int sum</span>(int a, int b, int <span style=color:purple;>c=1</span>) { return a+b+c; }
  };</code>

- Be careful with <span style=color:purple;>default values</span>. Here, the compiler cannot decide <b>unambiguously</b> which of the two functions should be used in the case <code>sum(arg0, arg1)</code>

## <center>Rule of thumb:</center>
<b>If it is not crystal clear to you which function should be used then the compiler will fail, too.</b>
<img src="plots/lecture3_StaticPolyRoT.png" width="300" >

## <center>Static polymorphism (master version1)</center>
<b>Method overloading</b>: decision about which method to call is made at compile time; hence the compiler can decide to <b>inline code</b> to improve performance (no overhead due to
function calls and copying of data to and from the stack!)

<code>std::cout &lt&lt statC.sum(1,2)   &lt&lt std::endl;
 std::cout &lt&lt statC.sum(1,2,3) &lt&lt std::endl;</code>
 
 becomes
 
 <code>std::cout &lt&lt (1+2)        &lt&lt std::endl;
 std::cout &lt&lt ((1+2)+3)    &lt&lt std::endl;</code>

## <center>Static polymorphism (master version2)</center>
<b>Method overloading</b>: since static polymorphism takes place at compile time, the <span style=color:red;>inline</span> specifier can be used to explicitly ‘suggest’ to the compiler to inline the function

<code>struct StaticCalcMaster {
    <span style=color:red;>inline</span> int sum(int a, int b)
      { return a+b; }
    <span style=color:red;>inline</span> int sum(int a, int b, int c)
      { return sum(sum(a,b),c); }
  };</code>

## <center>Dynamic polymorphism</center>
<b>Method overloading</b>: reimplementation of a function inherited
from a base class with new function body and same interface!

In [ ]:
struct BaseCalc {
    virtual int sum2(int a, int b) { return a+b; }
    int sum3(int a, int b, int c)
        { return sum2(sum2(a,b),c); }
  };
struct DerivedCalc : public BaseCalc {
    int sum2(int a, int b) override final { return b+a; }
};

In [2]:
int main() {
    BaseCalc baseCalc;
    DerivedCalc derivedCalc;
    // Using the base class to sum two numbers
    std::cout << "BaseCalc sum2: " << baseCalc.sum2(1, 2) << std::endl; 
    // Using the base class to sum three numbers
    std::cout << "BaseCalc sum3: " << baseCalc.sum3(1, 2, 3) << std::endl; 
    // Using the derived class to sum two numbers, this will use the overridden method
    std::cout << "DerivedCalc sum2: " << derivedCalc.sum2(1, 2) << std::endl;
    // Using the derived class to sum three numbers, this will use the base
    // class method sum3, which in turn will use the overridden sum2 method
    // from DerivedCalc
    std::cout << "DerivedCalc sum3: " << derivedCalc.sum3(1, 2, 3) << std::endl; 
    return 0;
}

SyntaxError: invalid syntax (1265796769.py, line 1)

## <center>Dynamic polymorphism(where things can go wrong)</center>
<b>Method overloading</b>: a common pitfall is to forget the virtual specifier in the base class to indicate that the <code>sum2</code> function from the base class can be overridden in a derived class

<code>struct BaseCalc {
    <span style=color:red;>int sum2</span>(int a, int b) { return a+b; }
    <span style=color:red;>int sum3</span>(int a, int b, int c){ return sum2(sum2(a,b),c); }
  };
  struct DerivedCalc : public BaseCalc {
    <span style=color:red;>int sum2</span>(int a, int b) { return b+a; }
};</code>

## <center>Dynamic polymorphism(where things can go wrong)</center>
Take home notes, read after the lecture.
Here's what happens if you forget <code>vitual</code> specifier
- <b>Method Hiding:</b> In <code>DerivedCalc</code>, the <code>sum2</code> method hides the <code>sum2</code> method from <code>BaseCalc</code>. This means that if you have an object of <code>DerivedCalc</code> and call <code>sum2</code>, it will use <code>DerivedCalc</code>'s <code>sum2</code>. However, if you have a pointer or reference to <code>BaseCalc</code> that actually points to a <code>DerivedCalc</code> object and call <code>sum2</code>, it will still use <code>BaseCalc</code>'s <code>sum2</code>. This is because the method is not virtual and hence does not support polymorphic behavior.


- <b>No Polymorphism:</b> Without the virtual keyword in <code>BaseCalc</code>, <code>sum2</code> in <code>DerivedCalc</code> doesn't exhibit polymorphic behavior, crucial for method overriding in C++.

<span style=color:red;><u>Rule of thumb:</u></span> These bugs are hard to find (they often remain unrecognised) and can be prevented by using the <code>override</code> keyword in C++11. It explicitly indicates that a function is intended to override a virtual function in a base class.

## <center>Dynamic polymorphism - general suggestions</center>
<b>Method overriding:</b> decision about which virtual method to call is made at run time; hence inlining is not possible

Common design pattern
- Specify <span style=color:red;>expected minimal functionality</span> of a group of classes in
abstract base class via <span style=color:red;>pure virtual member functions</span>
- Implement generic common functionality of a group of classes in
abstract base class via <span style=color:red;>virtual member functions</span>
- Implement specific functionality of a particular derived class by <span style=color:red;>overriding the pure virtual member function</span>

## <center>Example: inner product space</center>
In linear algebra, an inner product space is a vector space $V$ that is equipped with a special mapping (inner product)
$$\langle\cdot, \cdot\rangle: V \times V \rightarrow \mathbb{R}\text{ or }\mathbb{C}$$
Inner product spaces have a naturally induced norm
$$\|x\|=\sqrt{\langle x, x\rangle}$$

## <center>Example: inner product space</center>
Class <code>InnerProductSpaceBase</code> <b>declares inner product</b> as pure
virtual and <b>implements the naturally induced norm</b>

<code>struct InnerProductBase
{
    <span style=color:red;>virtual</span> double <span style=color:red;>inner_product</span>(... x,... y)<span style=color:red;> = 0</span>;
    double norm(... x) { return inner_product(x,x); } 
};</code>

Derived <code>InnerProductSpace</code> class <b>implements inner product</b>

<code>struct InnerProductSpace : public InnerProductSpaceBase {
    double <span style=color:red;>inner_product</span>(... x, ... y) = { return x*y; }};</code>

## <center>Implementation time! Task: Calculator2</center>
Extend the calculator class so that it can handle numbers of integer, float and double type at the same time

- Prevent manual code duplication 
- Prevent explicit type casting
- Make use of auto-functionality (C++11/14/17) 
- Make use of template meta programming

## <center>Vanilla implementation in C++</center>

In [ ]:
#include <iostream>
struct Calc2 {
    int sum(int a, int b) { return a + b; }
    int sum(int a, int b, int c) { return sum(sum(a, b), c); }
};
int main() {
    Calc2 calculator;
    // Using the sum method with two arguments
    std::cout << "Sum of 1 and 2: " << calculator.sum(1, 2) << std::endl;
    // Using the sum method with three arguments
    std::cout << "Sum of 1, 2, and 3: " << calculator.sum(1, 2, 3) << std::endl;
    return 0;
}

## <center>Automatic return type deduction (C++11)</center>
- Explicit definition of the function return type

<code><span style=color:blue;>int</span> sum(<span style=color:blue;>int</span> a, <span style=color:blue;>int</span> b){ return a+b; }</code>

- Automatic function return type (since C++11)

<code><span style=color:red;>auto</span> sum(<span style=color:blue;>int</span> a, <span style=color:blue;>int b)</span>  <span style=color:red;>-> decltype(a+b)</span>
      { return a+b; }</code>
- By using <span style=color:red;>decltype</span>, the <b>return type</b> of the sum function is determined automatically as the type of <span style=color:red;>operator+(a,b)</span>

## <center>Automatic return type deduction</center>
<span style=color:red;>decltype</span> specifier (C++11) queries the type of an expression

In [ ]:
#include <iostream>
struct Calc2 {
    auto sum(int a, int b) -> decltype(a + b) {return a + b;}
    auto sum(int a, int b, int c) -> decltype(sum(sum(a, b), c)) {
        return sum(sum(a, b), c);
    }
};

int main() {
    Calc2 C;
    std::cout << C.sum(1, 2) << std::endl;   // Outputs: 3
    std::cout << C.sum(1, 2, 3) << std::endl; // Outputs: 6
}

## <center>Automatic type deduction (C++14)</center>
C++14 deduces the type of parameters automatically

<code><span style=color:red;>auto</span> sum(int a, int b) <span style=color:blue;>// no -> decltype(...) needed</span>
    { return a+b; }
<span style=color:red;>auto</span> sum(int a, int b, int c) <span style=color:blue;>// no -> decltype(...) needed</span>
    { return sum(sum(a,b),c); }</code>
    
<u>Remark:</u> This C++14 feature helps to improve readability of the code and prevents deduction errors (due to forgotten/ inconsistent deduction rule by the programmer) but it does not solve the problem of being able to pass arguments of different types to the same function

## <center>Function templates</center>
Template meta programming is the standard technique to deal with arbitrary (= generic) function parameters

<b>Function templates:</b> allow you to implement so-called parameterized functions for generic parameter types

<code><span style=color:red;>template&lt</span><span style=color:purple;>typename R</span>, <span style=color:blue;>typename A, typename B</span>> 
<span style=color:purple;>R</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b)
{return a+b; }</code>

## <center>Function templates</center>
Types must be specified explicitly when the function is called

<code><span style=color:purple;>int</span> s1 = <span style=color:red;>sum&lt</span> <span style=color:purple;>int</span>, <span style=color:blue;>int, int</span>>(1, 2);
<span style=color:purple;>double</span> s2 = <span style=color:red;>sum&lt</span><span style=color:purple;>double</span>, <span style=color:blue;>double, int</span>>(1.2, 2); 
<span style=color:purple;>double</span> s3 = <span style=color:red;>sum&lt</span><span style=color:purple;>double</span>,<span style=color:blue;> float, double</span>>(1.4, 2.2);
</code>
This can be slightly simplified by using the <b>auto specifier</b>

<code><span style=color:purple;>auto</span> s1 = <span style=color:red;>sum&lt</span> <span style=color:purple;>int</span>, <span style=color:blue;>int, int</span>>(1, 2);
<span style=color:purple;>auto</span> s2 = <span style=color:red;>sum&lt</span><span style=color:purple;>double</span>, <span style=color:blue;>double, int</span>>(1.2, 2); 
<span style=color:purple;>auto</span> s3 = <span style=color:red;>sum&lt</span><span style=color:purple;>double</span>,<span style=color:blue;> float, double</span>>(1.4, 2.2);
</code>

## <center>Function templates</center>
C++11: <b>automatic return type deduction</b>

<code><span style=color:red;>template&lt</span> <span style=color:blue;>typename A, typename B</span>>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b)<span style=color:purple;> –> decltype(a+b)</span>{ return a+b; }</code>

C++14: <b>automatic type deduction</b>

<code><span style=color:red;>template&lt</span> <span style=color:blue;>typename A, typename B</span>>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b){ return a+b; }</code>

Usage

<code><span style=color:purple;>auto</span> s1 = <span style=color:red;>sum&lt</span><span style=color:blue;>int, int</span>>(1, 2);</code>

## <center>Function templates</center>
How to convert this function into a templated function

<code><span style=color:red;>int</span> sum(<span style=color:red;>int</span> a, <span style=color:red;>int</span> b, <span style=color:red;>int</span> c)
{
      return sum(sum(a,b), c);
  }</code>

## <center>Function templates</center>
Use explicit return type parameter (ugly!) -> NO
<code><span style=color:red;>template&lt</span><span style=color:purple;>typename R,</span><span style=color:blue;> typename A, typename B, typename C</span><span style=color:red;>></span>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b, <span style=color:blue;>C</span> c)
  {
      return <span style=color:red;>sum&lt</span><span style=color:purple;>R</span><span style=color:blue;>,C</span><span style=color:red;>></span>(<span style=color:red;>sum&lt</span><span style=color:blue;>A,B</span><span style=color:red;>></span>(a,b), c);
  }</code>
Guess what this function call will return

<code>auto s1 = sum<span style=color:red;>&lt</span> <span style=color:purple;>int,</span> <span style=color:blue;>double,double,double</span><span style=color:red;>></span>(1.1,2.2,3.3)</code>

## <center>Function templates</center>
Use a smart combination of templates and auto specifier

<code><span style=color:red;>template&lt</span><span style=color:blue;>typename A, typename B</span><span style=color:red;>></span>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b) <span style=color:purple;>-> decltype(a+b) // omit in C++14</span>
{return a+b; }
</code>

<code><span style=color:red;>template&lt</span><span style=color:blue;>typename A, typename B, typename C</span><span style=color:red;>></span> 
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b, <span style=color:blue;>C</span> c)
{
    return <span style=color:red;>sum&lt</span><span style=color:purple;>decltype(sum&ltA,B>(a,b))</span>,C<span style=color:red;>></span> (<span style=color:red;>sum&lt</span><span style=color:blue;>A,B</span><span style=color:red;>></span>(a,b), c);
}</code>

## <center>Function templates</center>
But wait, C++ <u>can</u> deduce the type of the template
argument from the given argument automatically

<code><span style=color:red;>template&lt</span><span style=color:blue;>typename A, typename B</span><span style=color:red;>></span>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b) <span style=color:purple;>-> decltype(a+b) // omit in C++14</span>
{return a+b; }
<span style=color:red;>template&lt</span><span style=color:blue;>typename A, typename B, typename C</span><span style=color:red;>></span> 
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b, <span style=color:blue;>C</span> c)
{
    return <span style=color:red;>sum</span>(<span style=color:red;>sum</span>(a,b), c);
}</code>

## <center>Function templates</center>
We can call the generic sum functions as follows:

<code><span style=color:purple;>auto</span> s1 = <span style=color:red;>sum&lt</span><span style=color:blue;>int, int</span><span style=color:red;>></span> (1, 2);
<span style=color:purple;>auto</span> s2 = <span style=color:red;>sum&lt</span><span style=color:blue;>double, int</span><span style=color:red;>></span>  (1.2, 2);
<span style=color:purple;>auto</span> s3 = <span style=color:red;>sum&lt</span><span style=color:blue;>float, double</span><span style=color:red;>></span>(1.4f, 2.2);</code>

Or, even simpler, as follows:

<code><span style=color:purple;>auto</span> s1 = <span style=color:red;>sum</span> (1, 2);
<span style=color:purple;>auto s2</span> = <span style=color:red;>sum</span> (1.2, 2);
<span style=color:purple;>auto s3</span> = <span style=color:red;>sum</span> (1.4f, 2.2)</code>

## <center>Function templates</center>
- Since the compiler needs to duplicate code and substitute A,B,C for each combination of templated types both the compile time and the size of the executable will increase
- 